In [12]:
from manim import *
import jupyter_capture_output

video_scene = " -v WARNING --progress_bar None --disable_caching et_Scene"
image_scene = f" -v WARNING --progress_bar None --disable_caching -r {2*427},{2*240}  -s et_Scene"

In [52]:
# Array mit Eye-Tracking Daten ([x, y, t])
et_array = np.array([
    [-6, -3, 200],
    [-5, 1, 400],
    [-4, 0, 600],
    [-3, 2, 200],
    [-2, -1, 300],
])

et_pos_array = np.zeros(np.shape(et_array))
et_pos_array[:,:2] = et_array[:,:2]

et_time_array = et_array[:,2] / 400

def polar_angle(v):
    return np.arctan(v[1]/v[0])

et_pos_array, et_time_array

(array([[-6., -3.,  0.],
        [-5.,  1.,  0.],
        [-4.,  0.,  0.],
        [-3.,  2.,  0.],
        [-2., -1.,  0.]]),
 array([0.5 , 1.  , 1.5 , 0.5 , 0.75]))

In [51]:
%%capture_video --path "animations/eye_tracking.mp4"
%%manim -ql --fps 20 $video_scene

class et_Scene(Scene):
    def construct(self):
        polarplane_pi = PolarPlane(
            azimuth_units = "degrees",
            size = 4,
            azimuth_label_font_size = 33.6,
            azimuth_step = 8,
            azimuth_label_buff = 0.2,
        ).add_coordinates(r_values= []).shift(4*RIGHT)
        self.add(polarplane_pi)


        for i, pos in enumerate(et_pos_array):
            if i == 0:
                dot = Dot(color = WHITE, radius = 0.1).move_to(pos)
                self.play(FadeIn(dot))
                self.wait(et_time_array[i])
            else:
                dot = Dot(color = WHITE, radius = 0.1).move_to(pos)
                arrow = Arrow(start = et_pos_array[i-1], end = pos, color = RED, stroke_width = 1, buff = 0.2, tip_length = 0.15)
                print(polar_angle(pos - et_pos_array[i-1]))
                self.play(FadeIn(dot), Create(arrow))
                self.wait(et_time_array[i])

1.3258176636680326
-0.7853981633974483
1.1071487177940904
-1.2490457723982544
